In [ ]:
import sys
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
import pandas as pd
import numpy as np
import glob, os

import itertools

from backtester.exoinfo import EXOInfo
from exobuilder.data.exostorage import EXOStorage
from IPython.core.display import display, HTML

import statsmodels.tsa.api as smt
import statsmodels.api as sm

import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go

import pandas as pd
import numpy as np
import scipy
import pprint

%pylab inline
figsize(20,10)

In [ ]:
# Loading global setting for MongoDB etc.
from scripts.settings import *

try:
    from scripts.settings_local import *
except:
    pass

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)
pp = pprint.PrettyPrinter(indent=4)

exo_list = storage.exo_list()
pp.pprint(exo_list)


df = pd.DataFrame()

for e in exo_list:
    exo_series, exo_dict = storage.load_series(e)
    
    exo_series = exo_series[exo_series.columns[0]]
    df[exo_dict['name']] = exo_series
#exo_series.plot()

In [ ]:
df.tail()

In [ ]:
px_ser = df.ES_BearishCollarBW
px_ser.asfreq('B').dropna().plot()

# Data examination

In [ ]:
#ts = px_ser.asfreq('B').dropna()
#ts = px_ser.ewm(2).mean().dropna() - px_ser.ewm(3).mean().dropna()
#ts = ts - ts.ewm(1).mean().dropna()
#ts = (px_ser - px_ser.ewm(10).mean()).dropna()
#ts = px_ser.diff(10).dropna()

#ts = (px_ser - px_ser.shift(1)).dropna()
#ts = ((px_ser - px_ser.ewm(1).mean()) / px_ser.ewm(1).std()).dropna()
ts = px_ser.rolling(10).mean().diff().dropna()

In [ ]:
#ts.cumsum().ix['2010':].plot()
ts.ix['2015':].plot(secondary_y=True)

#px_ser.ix['2010':].plot()
px_ser.ix['2015':].plot()

In [ ]:
smt.adfuller(ts)

In [ ]:
sm.graphics.tsa.plot_acf(ts, lags=70)

# Model

In [ ]:
i = 1

empty_ser = pd.Series(False, index=px_ser.dropna().index)

signals_df = pd.DataFrame()

signals_df['long_entry_signal'] = empty_ser
signals_df['long_exit_signal'] = empty_ser

signals_df['short_entry_signal'] = empty_ser
signals_df['short_exit_signal'] = empty_ser

while True:
    forecast_period = 2
    
    try:
        model = smt.ARIMA(ts.iloc[:i], order=(1,1,0)).fit()
    
    except Exception:
        i += forecast_period
        continue


    forecast_daterange = pd.date_range(start=ts.iloc[:i].index[-1] + BDay(1), periods=forecast_period)
    #forecast_daterange = pd.date_range(start=ts.iloc[:i].index[-1] + pd.DateOffset(1), freq='B')
    
    # Handling missing dates
    if (forecast_daterange.isin(px_ser.dropna().index).all() == False) & (forecast_daterange[-1] < ts.index[-1]):
        i += forecast_period
        print('DATES IS NOT EXIST', ts.iloc[:i].index[-4:])
        continue
    
    # If daterange is out of TSeries index, break while loop
    if forecast_daterange[-1] > ts.index[-1]:
        break
    
    forecast_values = model.forecast(forecast_period)

    forecast_series = pd.Series(forecast_values[0], index=forecast_daterange)
    
    if forecast_series[0] > forecast_series[-1]:
    #if forecast_series[0] > ts.iloc[:i][-1]:
        signals_df['long_entry_signal'].ix[ts.iloc[:i].index[-1]: 
                                           forecast_series.index[-1]] = True
        
        if (signals_df['long_entry_signal'].ix[forecast_series.index[-1]] == True):
            signals_df['long_exit_signal'].ix[forecast_series.index[-1] + BDay(1)] = True
            
           
    
    if forecast_series[0] < forecast_series[-1]:
    #if forecast_series[0] < ts.iloc[:i][-1]:
        signals_df['short_entry_signal'].ix[ts.iloc[:i].index[-1]: 
                                           forecast_series.index[-1]] = True
        
        if (signals_df['short_entry_signal'].ix[forecast_series.index[-1]] == True):
            signals_df['short_exit_signal'].ix[forecast_series.index[-1] + BDay(1)] = True
            
    print('last dates-', ts.iloc[:i].index[-3:])
    i += forecast_period
    
    
#signals_df.loc[(signals_df.long_exit_signal == True) & (signals_df.long_entry_signal == True), 'long_exit_signal'] = False
#signals_df.loc[(signals_df.short_exit_signal == True) & (signals_df.short_entry_signal == True), 'short_exit_signal'] = False

In [ ]:
entry = signals_df.long_entry_signal
exit = signals_df.long_exit_signal

#entry = signals_df.short_entry_signal
#exit = signals_df.short_exit_signal

exodf = pd.DataFrame()
exodf['exo'] = px_ser.dropna()

direction = -1
pl, inposition = backtester.backtest(exodf.iloc[:], entry, exit, direction)
equity, stats = backtester.stats(pl, inposition)

equity.plot()

(px_ser*-1).plot()

In [ ]:
(equity.diff() - 6).cumsum().plot()

In [ ]:
#stats

In [ ]:
#entry = signals_df.long_entry_signal
#exit = signals_df.long_exit_signal

entry = signals_df.short_entry_signal
exit = signals_df.short_exit_signal

exodf = pd.DataFrame()
exodf['exo'] = px_ser.asfreq('B').dropna()

direction = -1
pl, inposition = backtester.backtest(exodf.iloc[:], entry, exit, direction)
equity, stats = backtester.stats(pl, inposition)

equity.plot()

(px_ser*-1).plot()